# Poređenje rezultata

## CNN model treniran od nule

Tokom treniranja, vidi se da model brzo uči u prvim epohama – tačnost na treningu i validaciji raste do oko 0.86, a gubitak se stabilizuje nakon približno 10 epoha. Kriva validacione tačnosti je blizu trening krive, što ukazuje da nema značajnog overfittinga. Blago povećanje validacionog gubitka nakon 15. epohe pokazuje da je EarlyStopping ispravno zaustavio obuku u trenutku kada se model više nije poboljšavao.

<div style="text-align: center;">
  <img src="images/cnn.jpg" style="width: 600px; height: 300px">
</div>

Model treniran od nule pokazao je visoku tačnost i dobar balans između klasa, uz nešto veći loss koji ukazuje da su greške modela još uvijek prisutne kod složenijih uzoraka. Najveći broj grešaka odnosi se na lažno negativne slučajeve (59 slučajeva PNEUMONIA predviđeno kao NORMAL), što je u medicinskom kontekstu važan nalaz.

ROC kriva pokazuje solidnu sposobnost razlikovanja klasa, sa AUC vrijednošću od 0.977.

<div style="display: flex; gap: 50px;">

  <img src="images/cnn_confusion_matrix.png" alt="Confusion Matrix" width="400"/>
  <img src="images/cnn_roc.png" alt="ROC Curve" width="400"/>

</div>


## Transfer Learning - ResNet50 (Faza 1)

Na grafikonu se vidi da model vrlo brzo dostiže visoku tačnost u prvim epohama (iznad 0.8), ali validaciona kriva osciluje, što ukazuje na ograničenu sposobnost prilagođavanja — svi slojevi ResNet50 mreže bili su zamrznuti. Zbog toga model koristi unaprijed naučene karakteristike sa ImageNet skupa, ali ne uči dovoljno domensko-specifične obrasce karakteristične za rendgenske snimke pluća.


<div style="text-align: center;">
  <img src="images/resnet.jpg" style="width: 600px; height: 300px">
</div>

Iako loss djeluje nizak, test tačnost je manja nego kod CNN modela. Model značajno bolje prepoznaje PNEUMONIA klasu (recall 0.99), ali ima problem s razlikovanjem NORMAL snimaka (recall 0.43). Model ima tendenciju da prepoznaje više slika kao PNEUMONIA nego što zapravo jeste.

AUC rezultat od 0.953 potvrđuje visoku sposobnost razlikovanja između zdravih i oboljelih pacijenata, ali ipak nižu preciznost u odnosu na CNN. 
<div style="display: flex; gap: 50px;">

  <img src="images/resnet_confusion_matrix.png" alt="Confusion Matrix" width="400"/>
  <img src="images/resnet_roc.png" alt="ROC Curve" width="400"/>

</div>

## Transfer Learning - Fine-Tuned ResNet50 (Faza 2)

U drugoj fazi transfer learning pristupa, otključano je 30 najdubljih slojeva ResNet50 mreže, što je omogućilo da model fino prilagodi svoje težine rendgenskim slikama. Otključavanjem posljednjih slojeva mreže i dodatnim treniranjem sa manjom brzinom učenja, model uspijeva da se bolje prilagodi karakteristikama rendgenskih snimaka.

Rezultat pokazuje najveću tačnost, najniži test gubitak i najbolji balans između preciznosti i odziva.


<div style="text-align: center;">
  <img src="images/fine-tuned.jpg" style="width: 600px; height: 300px">
</div>

Fine-tuning je doveo do najboljih ukupnih rezultata. 
ROC kriva Fine-tuned modela dostiže AUC od 0.974 što je pokazatelj dobre sposobnosti razlikovanja klasa. 


<div style="display: flex; gap: 50px;">

  <img src="images/fine-tuned-confusion_matrix.png" alt="Confusion Matrix" width="400"/>
  <img src="images/fine-tuned-roc.png" alt="ROC Curve" width="400"/>

</div>

## Poređenje tri modela

<table style="width:100%; border-collapse: collapse; font-family: Arial, sans-serif; font-size: 14px; text-align: center;" border="1">
  <thead style="background-color: #4e5480ff; color: white;">
    <tr>
      <th style="padding: 10px;">Model</th>
      <th>Test Accuracy</th>
      <th>Test Loss</th>
      <th>AUC Score</th>
      <th>F1-score (NORMAL)</th>
      <th>F1-score (PNEUMONIA)</th>
      <th>Recall (NORMAL)</th>
      <th>Recall (PNEUMONIA)</th>
      <th>Opis</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td><strong>CNN</strong></td>
      <td>0.9192</td>
      <td>0.8639</td>
      <td><strong>0.977</strong></td>
      <td>0.86</td>
      <td>0.94</td>
      <td><strong>0.95</strong></td>
      <td>0.91</td>
      <td>stabilan model, dobro generalizuje, veći loss</td>
    </tr>
    <tr>
      <td><strong>ResNet</strong></td>
      <td>0.8350</td>
      <td>0.3125</td>
      <td>0.953</td>
      <td>0.58</td>
      <td>0.90</td>
      <td>0.43</td>
      <td><strong>0.99</strong></td>
      <td>brzo učenje, ali pristrasnost prema PNEUMONIA</td>
    </tr>
    <tr>
      <td><strong style="color: #626bf6ff;">Fine-Tuned</strong></td>
      <td><strong style="color: #626bf6ff;">0.9272</strong></td>
      <td><strong style="color: #626bf6ff;">0.1933</strong></td>
      <td><strong style="color: #626bf6ff;">0.974</strong></td>
      <td><strong style="color: #626bf6ff;">0.87</strong></td>
      <td><strong style="color: #626bf6ff;">0.95</strong></td>
      <td><strong style="color: #626bf6ff;">0.90</strong></td>
      <td><strong style="color: #626bf6ff;">0.94</strong></td>
      <td><strong style="color: #626bf6ff;">najbolji balans</strong></td>
    </tr>
  </tbody>
</table>


Važno je napomenuti da je Recall posebno značajna metrika u ovom kontekstu - ona mjeri osjetljivost modela, tj. koliko tačno model prepoznaje pozitivne slučajeve (pacijente sa pneumonijom).